In [64]:
import uproot
import awkward as ak
import numpy as np
import pandas as pd
from coffea import nanoevents
import vector
import fastjet

In [65]:
vector.register_awkward()

In [66]:
file = nanoevents.NanoEventsFactory.from_root(
    "filelist/badFromLuca/miniaod_1032.root",
    schemaclass=nanoevents.NanoAODSchema,
    #entry_stop=10000
)

In [67]:
events = file.events()

In [68]:
Jets = ak.zip(
    {
        "pt": events.Jet.pt,
        "eta": events.Jet.eta,
        "phi": events.Jet.phi,
        "mass": events.Jet.mass,
        "jetId": events.Jet.jetId,
    }
)

In [69]:
jetCut = (Jets.pt > 30) & (abs(Jets.eta) < 4.7)
ak4jets = Jets[jetCut]
ht = ak.sum(ak4jets.pt, axis=-1)

In [70]:
trigger_TripleMu = events.HLT.PFHT1050
events = events[trigger_TripleMu]
ak4jets = ak4jets[trigger_TripleMu]

In [71]:
events

<NanoEventsArray [<event 1032:1:118>, ... ] type='33 * event'>

In [72]:
Cands = ak.zip(
    {
        "pt": events.PFCands.trkPt,
        "eta": events.PFCands.trkEta,
        "phi": events.PFCands.trkPhi,
        "mass": events.PFCands.mass,
    },
    with_name="Momentum4D",
)

In [73]:
cut = (
    (events.PFCands.fromPV > 1)
    & (events.PFCands.trkPt >= 0.7)
    & (abs(events.PFCands.trkEta) <= 2.5)
    & (abs(events.PFCands.dz) < 10)
    & (events.PFCands.dzErr < 0.05)
)

In [74]:
Cleaned_cands = Cands[cut]
Cleaned_cands = ak.packed(Cleaned_cands)

In [75]:
LostTracks = ak.zip(
    {
        "pt": events.lostTracks.pt,
        "eta": events.lostTracks.eta,
        "phi": events.lostTracks.phi,
        "mass": 0.0,
    },
    with_name="Momentum4D",
)

In [76]:
cut = (
    (events.lostTracks.fromPV > 1)
    & (events.lostTracks.pt >= 0.7)
    & (abs(events.lostTracks.eta) <= 1.0)
    & (abs(events.lostTracks.dz) < 10)
    & (events.lostTracks.dzErr < 0.05)
)
Lost_Tracks_cands = LostTracks[cut]
Lost_Tracks_cands = ak.packed(Lost_Tracks_cands)

In [77]:
tracks = ak.concatenate([Cleaned_cands, Lost_Tracks_cands], axis=1)

In [78]:
print(tracks.pt)

[[0.757, 0.741, 0.786, 0.881, 0.828, 0.895, ... 2.86, 0.922, 1.48, 1.84, 1.86, 12.6]]


In [79]:
minPt = 150

In [80]:
jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.5)

In [81]:
cluster = fastjet.ClusterSequence(Cleaned_cands, jetdef)

In [82]:
ak_inclusive_jets = ak.with_name(cluster.inclusive_jets(min_pt=150), "Momentum4D")

In [83]:
ak_inclusive_cluster = ak.with_name(cluster.constituents(min_pt=150),"Momentum4D")

In [84]:
print(Cleaned_cands[0].pt)

[0.757, 0.741, 0.786, 0.881, 0.828, 0.895, ... 3.39, 1.02, 1.63, 0.964, 1.85, 1.22]


In [85]:
ak_inclusive_jets[0].pt

<Array [363, 506] type='2 * float64'>

In [86]:
minPtCut = ak_inclusive_jets.pt > minPt
ak_inclusive_jets = ak_inclusive_jets[minPtCut]
ak_inclusive_cluster = ak_inclusive_cluster[minPtCut]

In [87]:
tightJetId = ak4jets.jetId > 2
tight_ak4jets = ak4jets[tightJetId]
looseJetId = ak4jets.jetId >= 2
loose_ak4jets = ak4jets[looseJetId]

In [88]:
trackerCut = abs(ak4jets.eta) < 2.4
tracker_ak4jets = ak4jets[trackerCut]

In [89]:
from workflows import math_utils

In [90]:
indices = np.arange(0, len(tracks))
clusterCut = ak.num(ak_inclusive_jets, axis=1) > 1
ak_inclusive_cluster = ak_inclusive_cluster[clusterCut]
ak_inclusive_jets = ak_inclusive_jets[clusterCut]
tracks = tracks[clusterCut]
indices = indices[clusterCut]

In [94]:
len(tracks)

30

In [97]:
len(ak.num(tracks).to_list())

30

In [31]:
highpt_jet = ak.argsort(
    ak_inclusive_jets.pt, axis=1, ascending=False, stable=True
)
jets_pTsorted = ak_inclusive_jets[highpt_jet]
clusters_pTsorted = ak_inclusive_cluster[highpt_jet]

In [32]:
highpt_cands = clusters_pTsorted[:, 0]  # tracks for highest pt jet
singletrackCut = ak.num(highpt_cands) > 1
jets_pTsorted = jets_pTsorted[singletrackCut]
clusters_pTsorted = clusters_pTsorted[singletrackCut]
tracks = tracks[singletrackCut]
indices = indices[singletrackCut]

In [33]:
nconst_pTsorted = ak.num(clusters_pTsorted, axis=-1)

In [44]:
SUEP_cand = ak.where(
    nconst_pTsorted[:, 1] <= nconst_pTsorted[:, 0],
    jets_pTsorted[:, 0],
    jets_pTsorted[:, 1],
)
ISR_cand = ak.where(
    nconst_pTsorted[:, 1] > nconst_pTsorted[:, 0],
    jets_pTsorted[:, 0],
    jets_pTsorted[:, 1],
)
SUEP_cluster_tracks = ak.where(
    nconst_pTsorted[:, 1] <= nconst_pTsorted[:, 0],
    clusters_pTsorted[:, 0],
    clusters_pTsorted[:, 1],
)
ISR_cluster_tracks = ak.where(
    nconst_pTsorted[:, 1] > nconst_pTsorted[:, 0],
    clusters_pTsorted[:, 0],
    clusters_pTsorted[:, 1],
)

In [46]:
indices_CL = indices
SUEP_cand_CL = SUEP_cand
ISR_cand_CL = ISR_cand

In [48]:
boost_ISR = ak.zip(
    {
        "px": ISR_cand_CL.px * -1,
        "py": ISR_cand_CL.py * -1,
        "pz": ISR_cand_CL.pz * -1,
        "mass": ISR_cand_CL.mass,
    },
    with_name="Momentum4D",
)

In [50]:
ISR_tracks_b_CL = ISR_cluster_tracks.boost_p4(boost_ISR)

In [56]:
oneISRtrackCut = ak.num(ISR_tracks_b_CL) > 1

In [58]:
any(oneIRMtrackCut)

NameError: name 'oneIRMtrackCut' is not defined

In [35]:
cluster.constituents()

<MomentumArray4D [[[{pt: 0.713, ... mass: 0}]]] type='33 * var * var * Momentum4...'>

'0.8.5'